# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [1]:
# all import statements needed for the project, for example:
import os
import bs4
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import requests
import sqlalchemy as db
import re
from datetime import datetime, timedelta
import numpy as np
import fiona
import math
import pytest
from shapely.geometry import Point
from typing import Optional, Tuple, List

In [2]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = ""
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [8]:
def load_taxi_zones(file_path: str) -> GeoDataFrame:
    """
    Load taxi zones from a shapefile or GeoJSON file into a GeoDataFrame.

    Args:
        file_path (str): The path to the geospatial file (e.g., a shapefile or GeoJSON).

    Returns:
        GeoDataFrame: A GeoDataFrame containing the taxi zones data.
    """
    geofile: GeoDataFrame = gpd.read_file(file_path)
    return geofile

In [9]:
taxi_zones = load_taxi_zones("taxi_zones.shp")

In [10]:
def lookup_coords_for_taxi_zone_id(
    zone_loc_id: int, 
    loaded_taxi_zones: GeoDataFrame
) -> Optional[Tuple[float, float]]:
    """
    Look up the latitude and longitude coordinates for a taxi zone by its LocationID.

    Args:
        zone_loc_id (int): The LocationID of the taxi zone.
        loaded_taxi_zones (GeoDataFrame): A GeoDataFrame containing taxi zone geometries.

    Returns:
        Optional[Tuple[float, float]]: A tuple containing the latitude and longitude 
        coordinates if the zone is found, or None if the LocationID is not in the data.
    """
    # Ensure the CRS is set if missing
    if loaded_taxi_zones.crs is None:
        loaded_taxi_zones = loaded_taxi_zones.set_crs(epsg=2263)

    # Find the zone matching the LocationID
    zone = loaded_taxi_zones[loaded_taxi_zones['LocationID'] == zone_loc_id]
    if zone.empty:
        return None
    projected_zone = zone.to_crs(epsg=2263)
    centroid = projected_zone.geometry.centroid.iloc[0]
    centroid_geo = gpd.GeoSeries([centroid], crs=2263).to_crs(epsg=4326)

    # Return latitude and longitude as a tuple
    return (centroid_geo.geometry.iloc[0].y, centroid_geo.geometry.iloc[0].x)

### Calculate Sample Size

In [12]:
def calculate_sample_size(population: int, p: float = 0.5) -> int:
    """
    Calculate the required sample size using Cochran's formula, adjusted for finite populations.

    Args:
        population (int): The total population size.
        p (float, optional): Proportion of the population with the desired characteristic. 
            Defaults to 0.5 for maximum variability.

    Returns:
        int: The calculated sample size, rounded up to the nearest whole number.
    """
    # Z-value for 95% confidence level
    z: float = 1.96
    
    # Margin of error (5% by default)
    margin_of_error: float = 0.05
    
    # Complementary proportion
    q: float = 1 - p
    
    # Cochran's formula
    n_0: float = (z**2 * p * q) / (margin_of_error**2)
    
    # Adjust for finite population
    sample_size: float = n_0 / (1 + (n_0 - 1) / population)
    
    return math.ceil(sample_size)

### Common Functions

In [14]:
def get_all_urls_from_page(page_url: str) -> List[str]:
    """
    Fetch all URLs from a given webpage.

    Args:
        page_url (str): URL of the webpage to scrape.

    Returns:
        List[str]: A list of all URLs found on the webpage.

    Raises:
        Exception: If there is an error accessing the webpage or scraping its content.
    """
    try:
        response = requests.get(page_url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to access the URL: {page_url}. Error: {e}")
    
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a", href=True)
    all_urls: List[str] = [link["href"] for link in links]
    
    return all_urls

In [15]:
def filter_parquet_urls(links: List[str]) -> List[str]:
    """
    Filter a list of URLs to include only those pointing to Parquet files.

    Args:
        links (List[str]): A list of URLs to filter.

    Returns:
        List[str]: A list of URLs that point to Parquet files.
    """
    parquet_urls: List[str] = []
    for url in links:
        # Normalize the URL
        url = url.strip()
        if re.search(r"\.parquet(\?.*)?$", url):
            parquet_urls.append(url)
    
    return parquet_urls

### Process Taxi Data

In [17]:
def get_and_clean_taxi_month(parquet_url: str) -> pd.DataFrame:
    """
    Download, process, and sample a Yellow Taxi dataset for a given month.

    Args:
        parquet_url (str): URL of the Yellow Taxi Parquet file.

    Returns:
        pd.DataFrame: A sampled and processed DataFrame.
    """
    # Default directory for Yellow Taxi data
    save_dir = "processed_data/yellow_taxi"
    os.makedirs(save_dir, exist_ok=True)

    # Extract file name and define local path
    file_name = parquet_url.split("/")[-1]
    local_file_path = os.path.join(save_dir, file_name)

    # Download the file if not exist
    if not os.path.exists(local_file_path):
        response = requests.get(parquet_url, stream=True)
        response.raise_for_status()
        with open(local_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1MB chunks
                if chunk:
                    f.write(chunk)

    # Load the dataset
    data = pd.read_parquet(local_file_path)

    # Calculate sample size (using p = 0.5 for Yellow Taxi data)
    population = len(data)
    sample_size = calculate_sample_size(population, p = 0.5)
    sampled_data = data.sample(n=sample_size, random_state=42) if population > sample_size else data
    
    processed_file_path = os.path.join(save_dir, f"sampled_{file_name}")
    sampled_data.to_parquet(processed_file_path)
    return sampled_data


In [18]:
def get_and_clean_taxi_data(parquet_urls: List[str]) -> pd.DataFrame:
    """
    Download, process, and combine Yellow Taxi data from multiple Parquet file URLs.

    Args:
        parquet_urls (List[str]): A list of URLs pointing to Yellow Taxi Parquet files.

    Returns:
        pd.DataFrame: A combined DataFrame containing processed data from all valid Parquet files.
    """
    all_taxi_dataframes: List[pd.DataFrame] = []

    # Regex pattern for valid Yellow Taxi Parquet files (2020-2024 dates)
    yellow_taxi_pattern: re.Pattern = re.compile(
        r"yellow_tripdata_(2020-(0[1-9]|1[0-2])|202[1-3]-(0[1-9]|1[0-2])|2024-(0[1-8]))\.parquet"
    )

    # Filter URLs matching the pattern
    yellow_taxi_urls: List[str] = [url for url in parquet_urls if yellow_taxi_pattern.search(url)]

    for url in yellow_taxi_urls:
        save_dir: str = "processed_data/yellow_taxi"
        file_name: str = f"sampled_{url.split('/')[-1]}"
        processed_file_path: str = os.path.join(save_dir, file_name)

        if os.path.exists(processed_file_path):
            dataframe: pd.DataFrame = pd.read_parquet(processed_file_path)
        else:
            dataframe: pd.DataFrame = get_and_clean_taxi_month(url)

        all_taxi_dataframes.append(dataframe)

    # Combine all DataFrames into one
    taxi_data: pd.DataFrame = pd.concat(all_taxi_dataframes, ignore_index=True)

    return taxi_data

In [19]:
def clean_taxi_data(taxi_data: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the taxi data by retaining specific columns, normalizing column names,
    converting column types, and removing invalid trips. This includes unifying column names
    across different data sources using a predefined mapping.

    Args:
        taxi_data (pd.DataFrame): The input taxi data DataFrame.

    Returns:
        pd.DataFrame: The cleaned and filtered taxi data.
    """

    # Normalize column names using the column_mapping
    column_mapping = {
    'tpep_pickup_datetime': 'pickup_datetime',
    'tpep_dropoff_datetime': 'dropoff_datetime',
    'trip_distance': 'trip_miles'}
   
    taxi_data = taxi_data.rename(columns=column_mapping)

    
    # Add latitude and longitude
    taxi_data["PU_coords"] = taxi_data["PULocationID"].apply(lambda x: lookup_coords_for_taxi_zone_id(x, taxi_zones))
    taxi_data["DO_coords"] = taxi_data["DOLocationID"].apply(lambda x: lookup_coords_for_taxi_zone_id(x, taxi_zones))
    taxi_data = taxi_data.dropna(subset=["PU_coords", "DO_coords"]).reset_index(drop=True)
    taxi_data[["PU_lat", "PU_lon"]] = pd.DataFrame(taxi_data["PU_coords"].tolist(), index=taxi_data.index)
    taxi_data[["DO_lat", "DO_lon"]] = pd.DataFrame(taxi_data["DO_coords"].tolist(), index=taxi_data.index)
    taxi_data = taxi_data.drop(columns=["PU_coords", "DO_coords"])
    
    # Retain only the required columns
    required_columns = [
        'pickup_datetime', 'dropoff_datetime', 'trip_miles',
        'PU_lat', 'PU_lon', 'DO_lat', 'DO_lon', 'total_amount'
    ]
    taxi_data = taxi_data[required_columns]

    # Removing Invalid Data Points
    taxi_data = taxi_data[taxi_data["trip_miles"] > 0]
    taxi_data = taxi_data[taxi_data["total_amount"] > 0]
    taxi_data = taxi_data[taxi_data["pickup_datetime"] < taxi_data["dropoff_datetime"]]

    #Normalize column names
    taxi_data.columns = [col.lower() for col in taxi_data.columns]

    # Normalizing and Using Appropriate Column Types
    taxi_data["pickup_datetime"] = pd.to_datetime(taxi_data["pickup_datetime"], errors="coerce")
    taxi_data["dropoff_datetime"] = pd.to_datetime(taxi_data["dropoff_datetime"], errors="coerce")
    numeric_columns = ["trip_miles", "pu_lat", "pu_lon", "do_lat", "do_lon", "total_amount"]
    taxi_data[numeric_columns] = taxi_data[numeric_columns].apply(pd.to_numeric, errors="coerce")
    taxi_data = taxi_data.dropna(subset=["pickup_datetime", "dropoff_datetime"] + numeric_columns)

    # Removing Trips Outside the Latitude/Longitude Bounding Box
    lat_min, lon_min = 40.560445, -74.242330
    lat_max, lon_max = 40.908524, -73.717047
    taxi_data = taxi_data[
        (taxi_data["pu_lat"] >= lat_min) & (taxi_data["pu_lat"] <= lat_max) &
        (taxi_data["pu_lon"] >= lon_min) & (taxi_data["pu_lon"] <= lon_max) &
        (taxi_data["do_lat"] >= lat_min) & (taxi_data["do_lat"] <= lat_max) &
        (taxi_data["do_lon"] >= lon_min) & (taxi_data["do_lon"] <= lon_max)
    ]

    # Reset index after filtering
    taxi_data = taxi_data.reset_index(drop=True)

    return taxi_data

In [20]:
def get_taxi_data() -> pd.DataFrame:
    """
    Retrieve, filter, and clean taxi data from the TLC data page.

    This function performs the following steps:
    1. Fetches all URLs from the TLC webpage.
    2. Filters the URLs to retain only those pointing to Parquet files.
    3. Downloads, processes, and combines the data from these Parquet files.

    Returns:
        pd.DataFrame: A DataFrame containing the combined and cleaned taxi data.
    """
    all_urls: list[str] = get_all_urls_from_page(TLC_URL)
    all_parquet_urls: list[str] = filter_parquet_urls(all_urls)
    taxi_data: pd.DataFrame = get_and_clean_taxi_data(all_parquet_urls)

    return taxi_data

In [21]:
taxi_data = get_taxi_data()
taxi_data = clean_taxi_data(taxi_data)

In [22]:
taxi_data.head()

,pickup_datetime,dropoff_datetime,trip_miles,pu_lat,pu_lon,do_lat,do_lon,total_amount
0,2024-01-20 13:31:30,2024-01-20 14:03:25,17.14,40.646985,-73.786530,40.749914,-73.970443,90.96
1,2024-01-18 21:52:46,2024-01-18 22:03:21,2.49,40.764421,-73.977569,40.790011,-73.945750,22.50
2,2024-01-01 03:43:58,2024-01-01 03:50:47,1.84,40.866075,-73.919308,40.857779,-73.885867,12.50
3,2024-01-19 22:20:12,2024-01-19 22:50:12,3.60,40.748497,-73.992438,40.778766,-73.951010,33.95
4,2024-01-06 22:41:50,2024-01-06 22:43:24,0.04,40.791705,-73.973049,40.791705,-73.973049,6.20


In [23]:
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20727 entries, 0 to 20726
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   pickup_datetime   20727 non-null  datetime64[ns]
 1   dropoff_datetime  20727 non-null  datetime64[ns]
 2   trip_miles        20727 non-null  float64       
 3   pu_lat            20727 non-null  float64       
 4   pu_lon            20727 non-null  float64       
 5   do_lat            20727 non-null  float64       
 6   do_lon            20727 non-null  float64       
 7   total_amount      20727 non-null  float64       
dtypes: datetime64[ns](2), float64(6)
memory usage: 1.3 MB


In [24]:
taxi_data.describe()

,pickup_datetime,dropoff_datetime,trip_miles,pu_lat,pu_lon,do_lat,do_lon,total_amount
count,20727,20727,20727.000000,20727.000000,20727.000000,20727.000000,20727.000000,20727.000000
mean,2022-04-30 03:11:05.817195008,2022-04-30 03:27:08.452115712,3.271662,40.753350,-73.966955,40.755752,-73.970757,22.482544
min,2020-01-01 00:11:06,2020-01-01 00:30:50,0.010000,40.576961,-74.029893,40.576961,-74.174002,1.000000
25%,2021-02-27 15:22:24,2021-02-27 15:32:43.500000,1.090000,40.740337,-73.989845,40.740337,-73.989845,12.800000
50%,2022-04-29 09:06:52,2022-04-29 09:22:22,1.810000,40.758028,-73.977698,40.758028,-73.977698,17.020000
75%,2023-06-28 11:02:35,2023-06-28 11:17:36,3.310000,40.773633,-73.961764,40.775932,-73.959635,24.500000
max,2024-08-31 22:43:47,2024-08-31 23:26:23,67.900000,40.899528,-73.739337,40.899528,-73.726655,262.700000
std,NaN,NaN,4.110867,0.032385,0.045035,0.033130,0.036128,17.359922


In [25]:
output_file = "taxi_data_cleaned.csv"

# Save the DataFrame to a CSV file
taxi_data.to_csv(output_file, index=False)

### Processing Uber Data

In [27]:
def get_and_clean_uber_month(parquet_url: str) -> pd.DataFrame:
    """
    Download, process, and sample an Uber HVHF dataset for a given month.

    Args:
        parquet_url (str): URL of the Uber HVHF Parquet file.

    Returns:
        pd.DataFrame: A sampled and processed DataFrame.
    """
    # Define the directory for processed Uber HVHF data
    save_dir: str = "processed_data/hvhf"
    os.makedirs(save_dir, exist_ok=True)

    # Extract file name and define local path
    file_name: str = parquet_url.split("/")[-1]
    local_file_path: str = os.path.join(save_dir, file_name)

    # Download the file
    if not os.path.exists(local_file_path):
        try:
            response = requests.get(parquet_url, stream=True)
            response.raise_for_status()
            with open(local_file_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1MB chunks
                    if chunk:
                        f.write(chunk)
        except requests.exceptions.RequestException as e:
            raise Exception(f"Failed to download the file from {parquet_url}. Error: {e}")
    data: pd.DataFrame = pd.read_parquet(local_file_path)

    # Sample the dataset
    population: int = len(data)
    sample_size: int = calculate_sample_size(population, p=0.4)
    sampled_data: pd.DataFrame = data.sample(n=sample_size, random_state=42) if population > sample_size else data
    processed_file_path: str = os.path.join(save_dir, f"sampled_{file_name}")
    if not os.path.exists(processed_file_path):
        sampled_data.to_parquet(processed_file_path)

    return sampled_data

In [28]:
def get_and_clean_uber_data(parquet_urls: List[str]) -> pd.DataFrame:
    """
    Download, process, and combine Uber HVHF data from multiple Parquet file URLs.

    Args:
        parquet_urls (List[str]): A list of URLs pointing to Uber HVHF Parquet files.

    Returns:
        pd.DataFrame: A combined DataFrame containing processed data from all valid Parquet files.
    """
    all_uber_dataframes: List[pd.DataFrame] = []

    # Regex pattern to filter Uber HVHF Parquet files for specific dates (2020-2024)
    hvfhv_pattern: re.Pattern = re.compile(
        r"fhvhv_tripdata_(2020-(0[1-9]|1[0-2])|202[1-3]-(0[1-9]|1[0-2])|2024-(0[1-8]))\.parquet"
    )
    hvfhv_urls: List[str] = [url for url in parquet_urls if hvfhv_pattern.search(url)]

    for url in hvfhv_urls:
        save_dir: str = "processed_data/hvhf"
        file_name: str = f"sampled_{url.split('/')[-1]}"
        processed_file_path: str = os.path.join(save_dir, file_name)

        if os.path.exists(processed_file_path):
            dataframe: pd.DataFrame = pd.read_parquet(processed_file_path)
        else:
            dataframe: pd.DataFrame = get_and_clean_uber_month(url)

        all_uber_dataframes.append(dataframe)
    uber_data: pd.DataFrame = pd.concat(all_uber_dataframes, ignore_index=True)
    return uber_data

In [29]:
def load_and_clean_uber_data(uber_data: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and process Uber HVHF data, including filtering by license, adding coordinates,
    retaining required columns, and normalizing data.

    Args:
        uber_data (pd.DataFrame): The raw Uber HVHF data.

    Returns:
        pd.DataFrame: A cleaned and filtered DataFrame with necessary columns and normalized values.
    """
    # Filter by HVFH license number
    uber_data['hvfhs_license_num'] = uber_data['hvfhs_license_num'].astype(str)
    uber_data = uber_data[uber_data['hvfhs_license_num'] == 'HV0003'].copy()

    # Add coordinates for pickup and dropoff locations
    uber_data["PU_coords"] = uber_data["PULocationID"].apply(lambda x: lookup_coords_for_taxi_zone_id(x, taxi_zones))
    uber_data["DO_coords"] = uber_data["DOLocationID"].apply(lambda x: lookup_coords_for_taxi_zone_id(x, taxi_zones))
    uber_data = uber_data.dropna(subset=["PU_coords", "DO_coords"]).reset_index(drop=True)
    uber_data[["PU_lat", "PU_lon"]] = pd.DataFrame(uber_data["PU_coords"].tolist(), index=uber_data.index)
    uber_data[["DO_lat", "DO_lon"]] = pd.DataFrame(uber_data["DO_coords"].tolist(), index=uber_data.index)
    uber_data = uber_data.drop(columns=["PU_coords", "DO_coords"])

    # Retain required columns
    required_columns = [
        'pickup_datetime', 'dropoff_datetime', 'PU_lat', 'PU_lon', 
        'DO_lat', 'DO_lon', 'trip_miles', 'base_passenger_fare', 
        'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 
        'tips', 'airport_fee'
    ]
    uber_data = uber_data[required_columns]

    # Remove invalid data points
    uber_data = uber_data[uber_data["trip_miles"] > 0]
    uber_data = uber_data[uber_data["base_passenger_fare"] > 0]
    uber_data = uber_data[uber_data["pickup_datetime"] < uber_data["dropoff_datetime"]]

    # Normalize column names
    uber_data.columns = [col.lower() for col in uber_data.columns]

    # Ensure numeric columns are float or int
    numeric_columns = [
        'pu_lat', 'pu_lon', 'do_lat', 'do_lon', 'trip_miles', 
        'base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 
        'congestion_surcharge', 'tips', 'airport_fee'
    ]
    uber_data[numeric_columns] = uber_data[numeric_columns].apply(pd.to_numeric, errors="coerce")

    uber_data[numeric_columns] = uber_data[numeric_columns].fillna(0)
    uber_data = uber_data.dropna(subset=["pickup_datetime", "dropoff_datetime"])

    # Calculate the total amount
    uber_data['total_amount'] = (
        uber_data['base_passenger_fare'] + uber_data['tolls'] + 
        uber_data['bcf'] + uber_data['sales_tax'] + 
        uber_data['congestion_surcharge'] + uber_data['airport_fee'] + uber_data['tips']
    )

    # Retain only the updated columns
    updated_columns = [
        'pickup_datetime', 'dropoff_datetime', 'pu_lat', 
        'pu_lon', 'do_lat', 'do_lon', 'trip_miles', 'total_amount'
    ]
    uber_data = uber_data[updated_columns]

    # Filter trips within the latitude/longitude bounding box
    lat_min, lon_min = 40.560445, -74.242330
    lat_max, lon_max = 40.908524, -73.717047
    uber_data = uber_data[
        (uber_data["pu_lat"] >= lat_min) & (uber_data["pu_lat"] <= lat_max) &
        (uber_data["pu_lon"] >= lon_min) & (uber_data["pu_lon"] <= lon_max) &
        (uber_data["do_lat"] >= lat_min) & (uber_data["do_lat"] <= lat_max) &
        (uber_data["do_lon"] >= lon_min) & (uber_data["do_lon"] <= lon_max)
    ]

    # Reset index after filtering
    uber_data = uber_data.reset_index(drop=True)

    return uber_data

In [30]:
def get_uber_data() -> pd.DataFrame:
    """
    Retrieve, filter, process, and clean Uber HVHF data from the TLC data page.

    This function performs the following steps:
    1. Fetch all URLs from the TLC webpage.
    2. Filter the URLs to retain only those pointing to Parquet files.
    3. Download, process, and combine data from these Parquet files.
    4. Perform additional cleaning and normalization on the combined data.

    Returns:
        pd.DataFrame: A cleaned and processed DataFrame containing Uber HVHF trip data.
    """
    all_urls: list[str] = get_all_urls_from_page(TLC_URL)
    all_parquet_urls: list[str] = filter_parquet_urls(all_urls)
    uber_data: pd.DataFrame = get_and_clean_uber_data(all_parquet_urls)
    uber_data = load_and_clean_uber_data(uber_data)
    return uber_data

In [31]:
uber_data = get_uber_data()

In [32]:
uber_data.head()

,pickup_datetime,dropoff_datetime,pu_lat,pu_lon,do_lat,do_lon,trip_miles,total_amount
0,2024-01-26 08:07:17,2024-01-26 08:35:38,40.646116,-73.951623,40.666559,-73.895364,4.29,30.69
1,2024-01-19 02:17:05,2024-01-19 02:29:12,40.882403,-73.910665,40.857108,-73.932832,2.55,16.90
2,2024-01-21 01:44:00,2024-01-21 02:08:30,40.748575,-73.985156,40.715370,-73.936794,6.37,33.19
3,2024-01-20 12:58:40,2024-01-20 13:15:42,40.758028,-73.977698,40.753309,-74.004016,1.99,23.91
4,2024-01-02 08:40:48,2024-01-02 08:54:28,40.666559,-73.895364,40.676644,-73.913632,2.23,17.95


In [33]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14383 entries, 0 to 14382
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   pickup_datetime   14383 non-null  datetime64[ns]
 1   dropoff_datetime  14383 non-null  datetime64[ns]
 2   pu_lat            14383 non-null  float64       
 3   pu_lon            14383 non-null  float64       
 4   do_lat            14383 non-null  float64       
 5   do_lon            14383 non-null  float64       
 6   trip_miles        14383 non-null  float64       
 7   total_amount      14383 non-null  float64       
dtypes: datetime64[ns](2), float64(6)
memory usage: 899.1 KB


In [34]:
uber_data.describe()

,pickup_datetime,dropoff_datetime,pu_lat,pu_lon,do_lat,do_lon,trip_miles,total_amount
count,14383,14383,14383.000000,14383.000000,14383.000000,14383.000000,14383.000000,14383.000000
mean,2022-05-03 15:37:43.487311360,2022-05-03 15:55:31.077104896,40.737349,-73.935103,40.737331,-73.935269,4.372007,26.172938
min,2020-01-01 03:27:51,2020-01-01 03:30:21,40.561994,-74.170885,40.561994,-74.174002,0.010000,0.680000
25%,2021-02-27 15:08:56,2021-02-27 15:21:33,40.691201,-73.984197,40.691201,-73.984197,1.550000,12.855000
50%,2022-05-01 06:12:14,2022-05-01 06:17:35,40.736824,-73.948788,40.737698,-73.948136,2.830000,20.050000
75%,2023-07-02 21:51:33.500000,2023-07-02 22:17:46.500000,40.774376,-73.900317,40.774376,-73.899735,5.530000,32.370000
max,2024-08-31 21:37:05,2024-08-31 21:57:29,40.899528,-73.726655,40.899528,-73.726655,33.170000,239.830000
std,NaN,NaN,0.068382,0.064742,0.068795,0.068168,4.263149,19.989583


In [35]:
output_file = "uber_data_cleaned.csv"

# Save the DataFrame to a CSV file
uber_data.to_csv(output_file, index=False)

### Processing Weather Data

In [37]:
def get_all_weather_csvs(directory: str) -> List[str]:
    """
    Retrieve all CSV file paths from a specified directory.

    Args:
        directory (str): Path to the directory containing CSV files.

    Returns:
        List[str]: A list of file paths to all CSV files in the directory.
    """
    csv_files: List[str] = [
        os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".csv")
    ]
    return csv_files

In [38]:
def clean_month_weather_data_hourly(csv_file: str) -> pd.DataFrame:
    """
    Clean and process hourly weather data from a CSV file.

    Args:
        csv_file (str): Path to the CSV file containing weather data.

    Returns:
        pd.DataFrame: A DataFrame containing cleaned and processed hourly weather data.
                      Columns include 'date', 'precipitation', and 'wind_speed'.
                      Returns an empty DataFrame if the processing fails.
    """
    try:
        df: pd.DataFrame = pd.read_csv(csv_file, low_memory=False)
        df.rename(columns=lambda x: x.strip(), inplace=True)

        # Convert 'DATE' column to datetime
        df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")

        # Select and clean relevant columns
        hourly_data: pd.DataFrame = df[["DATE", "HourlyPrecipitation", "HourlyWindSpeed"]].copy()
        hourly_data.replace("T", 0, inplace=True)
        hourly_data.replace(regex=r"[^\d.]", value=np.nan, inplace=True)

        # Rename columns for consistency
        hourly_data.columns = ["date", "precipitation", "wind_speed"]

        # Convert columns to numeric
        hourly_data["precipitation"] = pd.to_numeric(hourly_data["precipitation"], errors="coerce")
        hourly_data["wind_speed"] = pd.to_numeric(hourly_data["wind_speed"], errors="coerce")

        return hourly_data

    except Exception as e:
        print(f"Error processing file {csv_file}: {e}")
        return pd.DataFrame()

In [39]:
def clean_month_weather_data_daily(csv_file: str) -> pd.DataFrame:
    """
    Clean and process daily weather data from a CSV file.

    Args:
        csv_file (str): Path to the CSV file containing weather data.

    Returns:
        pd.DataFrame: A DataFrame containing cleaned and processed daily weather data.
                      Columns include 'date', 'precipitation', 'average_wind_speed', and 'snowfall'.
    """
    try:
        df: pd.DataFrame = pd.read_csv(csv_file, low_memory=False)
        df.rename(columns=lambda x: x.strip(), inplace=True)

        # Convert 'DATE' column to datetime
        df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")

        # Select and clean relevant columns
        daily_data: pd.DataFrame = df[["DATE", "DailyPrecipitation", "DailyAverageWindSpeed", "DailySnowfall"]].copy()
        daily_data.replace("T", 0, inplace=True)
        daily_data.replace(regex=r"[^\d.]", value=np.nan, inplace=True)

        # Rename columns for consistency
        daily_data.columns = ["date", "precipitation", "average_wind_speed", "snowfall"]

        # Convert columns to numeric
        daily_data["precipitation"] = pd.to_numeric(daily_data["precipitation"], errors="coerce")
        daily_data["average_wind_speed"] = pd.to_numeric(daily_data["average_wind_speed"], errors="coerce")
        daily_data["snowfall"] = pd.to_numeric(daily_data["snowfall"], errors="coerce")

        # Drop rows for NaN
        daily_data = daily_data.dropna(subset=["precipitation", "average_wind_speed", "snowfall"], how="all")

        # Convert 'date' column to date
        daily_data["date"] = daily_data["date"].dt.date

        return daily_data

    except Exception as e:
        print(f"Error processing file {csv_file}: {e}")
        return pd.DataFrame()

In [40]:
def load_and_clean_weather_data(directory: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load and clean weather data from all CSV files in the specified directory.

    This function processes each file to extract and clean both hourly and daily weather data.

    Args:
        directory (str): Path to the directory containing CSV files.

    Returns:
        tuple: Two DataFrames:
            - hourly_data (pd.DataFrame): Combined cleaned hourly weather data.
            - daily_data (pd.DataFrame): Combined cleaned daily weather data.
    """
    weather_csv_files = get_all_weather_csvs(directory)
    hourly_dataframes = []
    daily_dataframes = []
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)

        # Append cleaned data to the respective lists
        if not hourly_dataframe.empty:
            hourly_dataframes.append(hourly_dataframe)
        if not daily_dataframe.empty:
            daily_dataframes.append(daily_dataframe)

    hourly_data = pd.concat(hourly_dataframes, ignore_index=True) if hourly_dataframes else pd.DataFrame()
    daily_data = pd.concat(daily_dataframes, ignore_index=True) if daily_dataframes else pd.DataFrame()
    
    # Replace NaN values with 0
    hourly_data.fillna(0, inplace=True)
    daily_data.fillna(0, inplace=True)
    
    return hourly_data, daily_data

In [41]:
if __name__ == "__main__":
    
    WEATHER_CSV_DIR = "weather/"  # Path to your weather data directory
    hourly_weather_data, daily_weather_data = load_and_clean_weather_data(WEATHER_CSV_DIR)

    hourly_weather_data.to_csv("cleaned_hourly_weather_data.csv", index=False)
    daily_weather_data.to_csv("cleaned_daily_weather_data.csv", index=False)

In [42]:
hourly_weather_data.head()

,date,precipitation,wind_speed
0,2020-01-01 00:51:00,0.0,8.0
1,2020-01-01 01:51:00,0.0,8.0
2,2020-01-01 02:51:00,0.0,14.0
3,2020-01-01 03:51:00,0.0,11.0
4,2020-01-01 04:51:00,0.0,6.0


In [43]:
hourly_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56098 entries, 0 to 56097
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           56098 non-null  datetime64[ns]
 1   precipitation  56098 non-null  float64       
 2   wind_speed     56098 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.3 MB


In [44]:
hourly_weather_data.describe()

,date,precipitation,wind_speed
count,56098,56098.000000,56098.000000
mean,2022-05-29 21:14:19.618881024,0.010288,4.537238
min,2020-01-01 00:51:00,0.000000,0.000000
25%,2021-03-18 19:01:45,0.000000,0.000000
50%,2022-05-28 01:21:00,0.000000,5.000000
75%,2023-08-15 05:39:00,0.000000,7.000000
max,2024-10-22 18:51:00,3.470000,2237.000000
std,NaN,0.056033,13.883208


In [45]:
daily_weather_data.head()

,date,precipitation,average_wind_speed,snowfall
0,2020-01-01,0.00,8.6,0.0
1,2020-01-02,0.00,5.4,0.0
2,2020-01-03,0.15,3.4,0.0
3,2020-01-04,0.27,4.4,0.0
4,2020-01-05,0.00,11.3,0.0


In [46]:
daily_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755 entries, 0 to 1754
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                1755 non-null   object 
 1   precipitation       1755 non-null   float64
 2   average_wind_speed  1755 non-null   float64
 3   snowfall            1755 non-null   float64
dtypes: float64(3), object(1)
memory usage: 55.0+ KB


In [47]:
daily_weather_data.describe()

,precipitation,average_wind_speed,snowfall
count,1755.000000,1755.000000,1755.000000
mean,0.141966,4.835499,0.039088
std,0.414574,2.467952,0.493457
min,0.000000,0.000000,0.000000
25%,0.000000,3.000000,0.000000
50%,0.000000,4.500000,0.000000
75%,0.060000,6.200000,0.000000
max,7.130000,14.200000,14.800000


## Part 2: Storing Cleaned Data

In [4]:
engine = db.create_engine(DATABASE_URL)

In [5]:
# Load cleaned datasets
daily_data = pd.read_csv("cleaned_daily_weather_data.csv")
hourly_data = pd.read_csv("cleaned_hourly_weather_data.csv")
taxi_data = pd.read_csv("taxi_data_cleaned.csv")
uber_data = pd.read_csv("uber_data_cleaned.csv")

In [6]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    datetime TEXT NOT NULL,
    temperature FLOAT,
    precipitation FLOAT,
    wind_speed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    date TEXT NOT NULL,
    avg_precipitation FLOAT,
    avg_wind_speed FLOAT,
    total_snowfall FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    pickup_datetime TEXT NOT NULL,
    dropoff_datetime TEXT NOT NULL,
    pu_lat FLOAT,
    pu_lon FLOAT,
    do_lat FLOAT,
    do_lon FLOAT,
    trip_miles FLOAT,
    total_amount FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    pickup_datetime TEXT NOT NULL,
    dropoff_datetime TEXT NOT NULL,
    pu_lat FLOAT,
    pu_lon FLOAT,
    do_lat FLOAT,
    do_lon FLOAT,
    trip_miles FLOAT,
    total_amount FLOAT
);
"""

In [7]:
# create that required schema.sql file
DATABASE_SCHEMA_FILE = "schema.sql"

with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [12]:
# create the tables with the schema files
with engine.connect() as connection:
    with open(DATABASE_SCHEMA_FILE, "r") as schema_file:
        schema_script = schema_file.read()
        statements = schema_script.split(";")
        for statement in statements:
            statement = statement.strip()
            if statement:
                connection.execute(db.text(statement))

### Add Data to Database

In [13]:
def write_dataframes_to_table(table_to_df_dict):
    """
    Writes dataframes to the corresponding SQL tables.
    Args:
        table_to_df_dict (dict): A dictionary where keys are table names
                                 and values are the respective DataFrames.
    """
    for table_name, df in table_to_df_dict.items():
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# Map table names to cleaned DataFrames
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

# Write data to the database
write_dataframes_to_table(map_table_name_to_dataframe)

In [14]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [15]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)